# Dataset_Collection

**Content**:
1. [Libraries](#libraries)
2. [Gridworld Creation](#gridworld)
3. [Functions](#funct)
4. [Non - Uniform Dataset Collection during Training Stage](#nonuniform)
5. [Uniform Dataset Collection During the Training Stage](#training_stage)
6. [Model Building](#transformations)
7. [Testing of Created Model](#test)

## 1. Libraries <a id = "libraries"> 

In [7]:
import numpy as np
import random
import sys
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

## 2. Gridworld Creation <a id = "gridworld">

In [8]:
def randPair(s,e):
    return np.random.randint(s,e), np.random.randint(s,e)

class BoardPiece:

    def __init__(self, name, code, pos):
        self.name = name #name of the piece
        self.code = code #an ASCII character to display on the board
        self.pos = pos #2-tuple e.g. (1,4)

class BoardMask:

    def __init__(self, name, mask, code):
        self.name = name
        self.mask = mask
        self.code = code

    def get_positions(self): #returns tuple of arrays
        return np.nonzero(self.mask)

def zip_positions2d(positions): #positions is tuple of two arrays
    x,y = positions
    return list(zip(x,y))

class GridBoard:

    def __init__(self, size=4):
        self.size = size #Board dimensions, e.g. 4 x 4
        self.components = {} #name : board piece
        self.masks = {}

    def addPiece(self, name, code, pos=(0,0)):
        newPiece = BoardPiece(name, code, pos)
        self.components[name] = newPiece

    #basically a set of boundary elements
    def addMask(self, name, mask, code):
        #mask is a 2D-numpy array with 1s where the boundary elements are
        newMask = BoardMask(name, mask, code)
        self.masks[name] = newMask

    def movePiece(self, name, pos):
        move = True
        for _, mask in self.masks.items():
            if pos in zip_positions2d(mask.get_positions()):
                move = False
        if move:
            self.components[name].pos = pos

    def delPiece(self, name):
        del self.components['name']

    def render(self):
        dtype = '<U2'
        displ_board = np.zeros((self.size, self.size), dtype=dtype)
        displ_board[:] = ' '

        for name, piece in self.components.items():
            displ_board[piece.pos] = piece.code

        for name, mask in self.masks.items():
            displ_board[mask.get_positions()] = mask.code

        return displ_board

    def render_np(self):
        num_pieces = len(self.components) + len(self.masks)
        displ_board = np.zeros((num_pieces, self.size, self.size), dtype=np.uint8)
        layer = 0
        for name, piece in self.components.items():
            pos = (layer,) + piece.pos
            displ_board[pos] = 1
            layer += 1

        for name, mask in self.masks.items():
            x,y = self.masks['boundary'].get_positions()
            z = np.repeat(layer,len(x))
            a = (z,x,y)
            displ_board[a] = 1
            layer += 1
        return displ_board

def addTuple(a,b):
    return tuple([sum(x) for x in zip(a,b)])

In [9]:
class Gridworld:

    def __init__(self, size=4, mode='static'):
        if size >= 4:
            self.board = GridBoard(size=size)
        else:
            print("Minimum board size is 4. Initialized to size 4.")
            self.board = GridBoard(size=4)

        #Add pieces, positions will be updated later
        self.board.addPiece('Player','P',(0,0))
        self.board.addPiece('Goal','+',(0,3))
        self.board.addPiece('Pit','-',(0,3))
        self.board.addPiece('Wall','W',(2,3))

        if mode == 'static':
            self.initGridStatic()
        elif mode == 'player':
            self.initGridPlayer()
        else:
            self.initGridRand()

    #Initialize stationary grid, all items are placed deterministically
    def initGridStatic(self):
        #Setup static pieces
        self.board.components['Player'].pos = randPair(0,self.board.size) #Row, Column
        self.board.components['Goal'].pos = (3,0)
        self.board.components['Pit'].pos = (3,1)
        self.board.components['Wall'].pos = (3,2)
        
        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridStatic()


    #Check if board is initialized appropriately (no overlapping pieces)
    #also remove impossible-to-win boards
    def validateBoard(self):
        valid = True

        player = self.board.components['Player']
        goal = self.board.components['Goal']
        wall = self.board.components['Wall']
        pit = self.board.components['Pit']

        all_positions = [piece for name,piece in self.board.components.items()]
        all_positions = [player.pos, goal.pos, wall.pos, pit.pos]
        if len(all_positions) > len(set(all_positions)):
            return False

        corners = [(0,0),(0,self.board.size), (self.board.size,0), (self.board.size,self.board.size)]
        #if player is in corner, can it move? if goal is in corner, is it blocked?
        if player.pos in corners or goal.pos in corners:
            val_move_pl = [self.validateMove('Player', addpos) for addpos in [(0,1),(1,0),(-1,0),(0,-1)]]
            val_move_go = [self.validateMove('Goal', addpos) for addpos in [(0,1),(1,0),(-1,0),(0,-1)]]
            if 0 not in val_move_pl or 0 not in val_move_go:
                #print(self.display())
                #print("Invalid board. Re-initializing...")
                valid = False

        return valid

    #Initialize player in random location, but keep wall, goal and pit stationary
    def initGridPlayer(self):
        #height x width x depth (number of pieces)
        self.initGridStatic()
        #place player
        self.board.components['Player'].pos = randPair(0,self.board.size)

        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridPlayer()

    #Initialize grid so that goal, pit, wall, player are all randomly placed
    def initGridRand(self):
        #height x width x depth (number of pieces)
        self.board.components['Player'].pos = randPair(0,self.board.size)
        self.board.components['Goal'].pos = randPair(0,self.board.size)
        self.board.components['Pit'].pos = randPair(0,self.board.size)
        self.board.components['Wall'].pos = randPair(0,self.board.size)

        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridRand()

    def validateMove(self, piece, addpos=(0,0)):
        outcome = 0 #0 is valid, 1 invalid, 2 lost game
        pit = self.board.components['Pit'].pos
        wall = self.board.components['Wall'].pos
        new_pos = addTuple(self.board.components[piece].pos, addpos)
        if new_pos == wall:
            outcome = 1 #block move, player can't move to wall
        elif max(new_pos) > (self.board.size-1):    #if outside bounds of board
            outcome = 1
        elif min(new_pos) < 0: #if outside bounds
            outcome = 1
        elif new_pos == pit:
            outcome = 2

        return outcome

    def makeMove(self, action):
        #need to determine what object (if any) is in the new grid spot the player is moving to
        #actions in {u,d,l,r}
        def checkMove(addpos):
            if self.validateMove('Player', addpos) in [0,2]:
                new_pos = addTuple(self.board.components['Player'].pos, addpos)
                self.board.movePiece('Player', new_pos)

        if action == 'u': #up
            checkMove((-1,0))
        elif action == 'd': #down
            checkMove((1,0))
        elif action == 'l': #left
            checkMove((0,-1))
        elif action == 'r': #right
            checkMove((0,1))
        else:
            pass

    def reward(self):
        if (self.board.components['Player'].pos == self.board.components['Pit'].pos):
            return -10
        elif (self.board.components['Player'].pos == self.board.components['Goal'].pos):
            return 10
        else:
            return -1

    def display(self):
        return self.board.render()

In [10]:
action_set = {
    0: 'u',
    1: 'd',
    2: 'l',
    3: 'r',
}

## 3. Functions <a id = 'funct'> 

In [11]:
"""
from2dto1d:
    Args: 
        pos(tuple):2d position of the objects(Player, Wall, Goal, Pit) in the gridworld
    Return:
        return(integer): 1d position of the objects(Player, Wall, Goal, Pit)in the gridworld
""" 
def from2dto1d(pos):
    if pos == '(0, 0)':
        return 0
    if pos == '(0, 1)':
        return 1
    if pos == '(0, 2)':
        return 2
    if pos == '(0, 3)':
        return 3
    if pos == '(1, 0)':
        return 4
    if pos == '(1, 1)':
        return 5
    if pos == '(1, 2)':
        return 6
    if pos == '(1, 3)':
        return 7
    if pos == '(2, 0)':
        return 8
    if pos == '(2, 1)':
        return 9
    if pos == '(2, 2)':
        return 10
    if pos == '(2, 3)':
        return 11
    if pos == '(3, 0)':
        return 12
    if pos == '(3, 1)':
        return 13
    if pos == '(3, 2)':
        return 14
    if pos == '(3, 3)':
        return 15

In [12]:
def from1dto2d(pos):
    if pos == 0:
        return (0, 0)
    if pos == 1:
        return (0, 1)
    if pos == 2:
        return (0, 2)
    if pos == 3:
        return (0, 3)
    if pos == 4:
        return (1, 0)
    if pos == 5:
        return (1, 1)
    if pos == 6:
        return (1, 2)
    if pos == 7:
        return (1, 3)
    if pos == 8:
        return (2, 0)
    if pos == 9:
        return (2, 1)
    if pos == 10:
        return (2, 2)
    if pos == 11:
        return (2, 3)
    if pos == 12:
        return (3, 0)
    if pos == 13:
        return (3, 1)
    if pos == 14:
        return (3, 2)
    if pos == 15:
        return (3, 3)

## 3. Model Building <a id = 'transformations'> 

In [17]:
import numpy as np
import torch
from IPython.display import clear_output
import random
from matplotlib import pylab as plt

l1 = 64
l2 = 200
l3 = 120
l4 = 4

model = torch.nn.Sequential(
    torch.nn.Linear(l1, l2),
    torch.nn.ReLU(),
    torch.nn.Linear(l2, l3),
    torch.nn.ReLU(),
    torch.nn.Linear(l3,l4)
)
loss_fn = torch.nn.MSELoss()
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

gamma = 0.9
epsilon = 0.3

In [18]:
from collections import deque
epochs = 5000
losses = []
mem_size = 1000 #A
batch_size = 200 #B
replay = deque(maxlen=mem_size) #C
max_moves = 50 #D

counter = 0
num = 0
for i in range(epochs):
    game = Gridworld(size=4, mode='random')
    state1_ = game.board.render_np().reshape(1,64) + np.random.rand(1,64)/100.0
    state1 = torch.from_numpy(state1_).float()
    status = 1
    mov = 0
    while(status == 1): 
        mov += 1
        qval = model(state1) #E
        qval_ = qval.data.numpy()
            
        if (random.random() < epsilon): #F
            action_ = np.random.randint(0,4)
        else:
            action_ = np.argmax(qval_)
        
        action = action_set[action_]
        
        
        game.makeMove(action)
        state2_ = game.board.render_np().reshape(1,64) + np.random.rand(1,64)/100.0
        state2 = torch.from_numpy(state2_).float()
        next_state = model(state2)
        next_state_ = next_state.data.numpy()
        
        reward = game.reward()
        if reward == -1 or reward == -10: #N
            Y = reward + (gamma * next_state_[0][np.argmax(next_state_)])
        else:
            Y = reward
        done = True if reward > 0 else False
        exp =  (state1, action_, reward, state2, done) #G
        replay.append(exp) #H
        state1 = state2
        
        
        
        if len(replay) > batch_size: #I
            minibatch = random.sample(replay, batch_size) #J
            state1_batch = torch.cat([s1 for (s1,a,r,s2,d) in minibatch]) #K
            action_batch = torch.Tensor([a for (s1,a,r,s2,d) in minibatch])
            reward_batch = torch.Tensor([r for (s1,a,r,s2,d) in minibatch])
            state2_batch = torch.cat([s2 for (s1,a,r,s2,d) in minibatch])
            done_batch = torch.Tensor([d for (s1,a,r,s2,d) in minibatch])
            
            Q1 = model(state1_batch) #L
            with torch.no_grad():
                Q2 = model(state2_batch) #M
            
            Y = reward_batch + gamma * ((1 - done_batch) * torch.max(Q2,dim=1)[0]) #N
            X = Q1.gather(dim=1,index=action_batch.long().unsqueeze(dim=1)).squeeze()
            loss = loss_fn(X, Y.detach())
            print(i)
            clear_output(wait=True)
            optimizer.zero_grad()
            loss.backward()
            losses.append(loss.item())
            optimizer.step()
        counter += 1
        if reward != -1 or mov > max_moves: #O
            status = 0
            mov = 0
losses = np.array(losses)

4999


## 4. Non - Uniform Dataset Collection during Training Stage  <a id = 'nonuniform'> 

In [19]:
from collections import deque
epochs = 5000
losses = []
mem_size = 1000 #A
batch_size = 200 #B
replay = deque(maxlen=mem_size) #C
max_moves = 50 #D

h = 0
w, h = 20, 300000
matrix = [[0 for x in range(w)] for y in range(h)] 

counter = 0
num = 0
for i in range(epochs):
    game = Gridworld(size=4, mode='random')
    state1_ = game.board.render_np().reshape(1,64) + np.random.rand(1,64)/100.0
    state1 = torch.from_numpy(state1_).float()
    status = 1
    mov = 0
    while(status == 1): 
        mov += 1
        qval = model(state1) #E
        qval_ = qval.data.numpy()
        
        matrix[counter][0] = i + 5000
        matrix[counter][1] = from2dto1d(str(game.board.components['Player'].pos))
        matrix[counter][2] = from2dto1d(str(game.board.components['Pit'].pos))
        matrix[counter][3] = from2dto1d(str(game.board.components['Goal'].pos))
        matrix[counter][4] = from2dto1d(str(game.board.components['Wall'].pos))
        matrix[counter][5] = action_
                        
        matrix[counter][9] = qval_[0][np.argmax(qval_)]
        matrix[counter][10] = qval_[0][0]
        matrix[counter][11] = qval_[0][1]
        matrix[counter][12] = qval_[0][2]
        matrix[counter][13] = qval_[0][3]
                        
            
        if (random.random() < epsilon): #F
            action_ = np.random.randint(0,4)
            matrix[counter][8] = 1 
        else:
            action_ = np.argmax(qval_)
            matrix[counter][8] = 0 
        
        action = action_set[action_]
        
        
        game.makeMove(action)
        state2_ = game.board.render_np().reshape(1,64) + np.random.rand(1,64)/100.0
        state2 = torch.from_numpy(state2_).float()
        next_state = model(state2)
        next_state_ = next_state.data.numpy()
        
        matrix[counter][18] = next_state_[0][np.argmax(next_state_)]
        matrix[counter][14] = next_state_[0][0]
        matrix[counter][15] = next_state_[0][1]
        matrix[counter][16] = next_state_[0][2]
        matrix[counter][17] = next_state_[0][3]
        
        reward = game.reward()
        matrix[counter][6] = reward
        if reward == -1 or reward == -10: #N
            Y = reward + (gamma * next_state_[0][np.argmax(next_state_)])
        else:
            Y = reward
        matrix[counter][19] = Y  
        done = True if reward > 0 else False
        matrix[counter][7] = from2dto1d(str(game.board.components['Player'].pos))
        exp =  (state1, action_, reward, state2, done) #G
        replay.append(exp) #H
        state1 = state2
        
        
        
        if len(replay) > batch_size: #I
            minibatch = random.sample(replay, batch_size) #J
            state1_batch = torch.cat([s1 for (s1,a,r,s2,d) in minibatch]) #K
            action_batch = torch.Tensor([a for (s1,a,r,s2,d) in minibatch])
            reward_batch = torch.Tensor([r for (s1,a,r,s2,d) in minibatch])
            state2_batch = torch.cat([s2 for (s1,a,r,s2,d) in minibatch])
            done_batch = torch.Tensor([d for (s1,a,r,s2,d) in minibatch])
            
            Q1 = model(state1_batch) #L
            with torch.no_grad():
                Q2 = model(state2_batch) #M
            
            Y = reward_batch + gamma * ((1 - done_batch) * torch.max(Q2,dim=1)[0]) #N
            X = Q1.gather(dim=1,index=action_batch.long().unsqueeze(dim=1)).squeeze()
            loss = loss_fn(X, Y.detach())
            print(i)
            clear_output(wait=True)
            optimizer.zero_grad()
            loss.backward()
            losses.append(loss.item())
            optimizer.step()
        counter += 1
        if reward != -1 or mov > max_moves: #O
            status = 0
            mov = 0
losses = np.array(losses)

4999


In [24]:
import pandas as pd
column_names = ["Epochs","Player", "Pit", "Goal", "Wall", "Action",  "Reward", "Next_State", "Epsilon_Boolean", 
                "Q_MAX",  "Q1_UP", "Q2_DOWN", "Q3_LEFT", "Q4_RIGHT",  "Q1_UP_NEXT", "Q2_DOWN_NEXT", "Q3_LEFT_NEXT", "Q4_RIGHT_NEXT","Q_MAX_NEXT", "Y"
                ]
df = pd.DataFrame(matrix, columns = column_names)
df = df.loc[(df != 0).any(axis=1)]
df

,Epochs,Player,Pit,Goal,Wall,Action,Reward,Next_State,Epsilon_Boolean,Q_MAX,Q1_UP,Q2_DOWN,Q3_LEFT,Q4_RIGHT,Q1_UP_NEXT,Q2_DOWN_NEXT,Q3_LEFT_NEXT,Q4_RIGHT_NEXT,Q_MAX_NEXT,Y
0,5000,1,2,7,3,3,-10,2,0,3.510271,1.555545,2.999122,2.275227,3.510271,4.184848,5.752448,3.376853,4.921612,5.752448,-4.822797
1,5001,10,5,4,1,3,-1,9,0,5.080411,3.836540,3.423163,5.080411,0.274622,0.780820,3.465996,6.651873,2.987756,6.651873,4.986685
2,5001,9,5,4,1,2,-1,8,0,6.651873,0.780820,3.465996,6.651873,2.987756,9.241036,7.959845,7.022808,4.492457,9.241036,7.316933
3,5001,8,5,4,1,2,10,4,0,9.241036,9.241036,7.959845,7.022808,4.492457,7.231631,9.194542,6.686027,2.019725,9.194542,10.000000
4,5002,8,6,11,15,0,-1,9,0,6.946158,5.624981,4.882193,1.529516,6.946158,5.090035,6.036646,3.182453,9.009235,9.009235,7.108312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23271,9997,2,1,10,7,1,-1,6,1,7.669364,5.056441,7.669364,4.241444,2.276922,6.645481,9.331314,7.377896,4.460841,9.331314,7.398183
23272,9997,6,1,10,7,1,10,10,0,9.346166,6.648674,9.346166,7.398812,4.500574,8.230506,7.491100,8.188928,6.278389,8.230506,10.000000
23273,9998,8,12,13,1,1,-1,9,0,7.284459,3.953820,4.577577,6.689157,7.284459,7.070791,10.849091,7.721227,6.519612,10.849091,8.764182
23274,9998,9,12,13,1,3,10,13,0,10.839301,7.044550,10.839301,7.724490,6.530786,7.465142,8.581019,7.485756,7.726900,8.581019,10.000000


In [25]:
df.to_csv('DatasetBeforeNonUniform.csv')

## 6. Testing of Created Model <a id = 'test'> 

In [8]:
def dqn_test_model(model, mode='static', display=True):
    global counter
    i = 0
    test_game = Gridworld(mode=mode)
    state_ = test_game.board.render_np().reshape(1,64) + np.random.rand(1,64)/10.0
    state = torch.from_numpy(state_).float()
    if display:
        print("Initial State:")
        print(test_game.display())
    status = 1
    while(status == 1): #A
        qval = model(state)
        qval_ = qval.data.numpy()
        action_ = np.argmax(qval_) #B
        action = action_set[action_]
        p_curr = from2dto1d((str(test_game.board.components['Player'].pos)))
        pi = from2dto1d((str(test_game.board.components['Pit'].pos)))
        g = from2dto1d((str(test_game.board.components['Goal'].pos)))
        w = from2dto1d((str(test_game.board.components['Wall'].pos)))
        if display:
            print('Move #: %s; Taking action: %s' % (i, action))
        test_game.makeMove(action)
        p_next = from2dto1d((str(test_game.board.components['Player'].pos)))

        state_ = test_game.board.render_np().reshape(1,64) + np.random.rand(1,64)/10.0
        state = torch.from_numpy(state_).float()
        if display:
            print(test_game.display())
        reward = test_game.reward()
        if reward != -1:
            if reward > 0:
                status = 2
                if display:
                    print("Game won! Reward: %s" % (reward,))
            else:
                status = 0
                if display:
                    print("Game LOST. Reward: %s" % (reward,))
        i += 1
        counter += 1
        if (i > 15):
            if display:
                print("Game lost; too many moves.")
            break
    
    win = True if status == 2 else False
    return win

In [9]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = dqn_test_model(model, random, display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num /10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 906
0.906
0.906
Games played: 1000, # of wins: 901
0.901
1.807
Games played: 1000, # of wins: 910
0.91
2.717
Games played: 1000, # of wins: 915
0.915
3.632
Games played: 1000, # of wins: 894
0.894
4.526
Games played: 1000, # of wins: 927
0.927
5.452999999999999
Games played: 1000, # of wins: 900
0.9
6.353
Games played: 1000, # of wins: 907
0.907
7.26
Games played: 1000, # of wins: 899
0.899
8.158999999999999
Games played: 1000, # of wins: 905
0.905
9.063999999999998
Win percentage: 90.63999999999999%


## 7. Uniform Dataset Collection from Trained Model <a id = 'uniform'> 

In [10]:
h = 0
w, h = 9, 100000
matrix = [[0 for x in range(w)] for y in range(h)] 

counter = 0
num = 0
epoch = 0
epsilon = 0
for i in range(0,16):
    for j in range(0,16):
        for k in range (0,16):
            for c in range (0,16):
                if  c != j and k != i and k!= j and i != j:
                    game = Gridworld(size=4, mode='random')
                    game.board.components['Player'].pos = from1dto2d(c)
                    game.board.components['Goal'].pos = from1dto2d(k)
                    game.board.components['Wall'].pos = from1dto2d(j)
                    game.board.components['Pit'].pos = from1dto2d(i)
                    state1_ = game.board.render_np().reshape(1,64) + np.random.rand(1,64)/100.0
                    state1 = torch.from_numpy(state1_).float()
                    status = 1
                    qval = model(state1) #E
                    qval_ = qval.data.numpy()

                    matrix[counter][5] = qval_[0][0]
                    matrix[counter][6] = qval_[0][1]
                    matrix[counter][7] = qval_[0][2]
                    matrix[counter][8] = qval_[0][3]

                    if (random.random() < epsilon): #F
                        action_ = np.random.randint(0,4)
                    else:
                        action_ = np.argmax(qval_)

                    action = action_set[action_]

                    matrix[counter][0] = from2dto1d(str(game.board.components['Player'].pos))
                    matrix[counter][1] = from2dto1d(str(game.board.components['Pit'].pos))
                    matrix[counter][2] = from2dto1d(str(game.board.components['Goal'].pos))
                    matrix[counter][3] = from2dto1d(str(game.board.components['Wall'].pos))
                    
                    reward = game.reward()
                    matrix[counter][4] = reward
                    
                    game.makeMove(action)
                    state2_ = game.board.render_np().reshape(1,64) + np.random.rand(1,64)/100.0
                    state2 = torch.from_numpy(state2_).float()
                    next_state = model(state2)
                    next_state_ = next_state.data.numpy()

                    reward = game.reward()
                    done = True if reward > 0 else False
                    state1 = state2
                    print(epoch)
                    clear_output(wait=True)
                    counter += 1
                    epoch += 1

50399


In [12]:
import pandas as pd
column_names = ["Current_State", "Pit_Position", "Goal_Position", "Wall_Position",  "Current_Reward", 
                 "Q1_UP", "Q2_DOWN", "Q3_LEFT", "Q4_RIGHT"
                ]
df = pd.DataFrame(matrix, columns = column_names)
df = df.loc[(df != 0).any(axis=1)]
df

,Current_State,Pit_Position,Goal_Position,Wall_Position,Current_Reward,Q1_UP,Q2_DOWN,Q3_LEFT,Q4_RIGHT
0,0,0,2,1,-10,2.614309,2.558848,3.948454,4.814851
1,2,0,2,1,10,6.871885,6.282998,6.243661,6.415882
2,3,0,2,1,-1,7.459104,5.998907,8.490587,6.589260
3,4,0,2,1,-1,0.305944,1.363386,3.014956,4.630818
4,5,0,2,1,-1,3.776497,4.053163,3.613774,8.103388
...,...,...,...,...,...,...,...,...,...
50395,10,15,13,14,-1,3.689109,7.530389,7.412851,6.018137
50396,11,15,13,14,-1,1.637386,3.614807,5.358669,4.984922
50397,12,15,13,14,-1,5.306869,7.611459,9.131452,12.008292
50398,13,15,13,14,10,7.040803,8.193069,7.585652,8.199212


In [14]:
df.to_csv('DatasetBeforeUniform.csv')